## Table of Contents

### 1.  Deploying Transcript Generation model using Flask
> #### 1.1 Install required libraries
> #### 1.2 Setup Front-end
> #### 1.3 Setup Backend and Deploy it using Flask

---

### 1.1 Install required libraries

In [1]:
# install important libraries
!pip install -q flask-ngrok
!pip install -q omegaconf soundfile torchaudio

     |████████████████████████████████| 1.9MB 19.0MB/s 
     |████████████████████████████████| 645kB 43.7MB/s 


### 1.2 Setup Front-end

In [2]:
# create folder for HTML/CSS files
!mkdir templates

In [3]:
## create home page for website
%%writefile templates/index.html

<!DOCTYPE html>
<html lang="en">
    <head>
        <meta charset="UTF-8" />
        <title>Transcript Generator</title>
    </head>
    <body>
        <h1>Upload Video File</h1>
        <form method=post enctype=multipart/form-data>
            <input type=file name=file>
            <input type=submit value=Upload>
        </form>
        <h1>Export Transcript</h1>
        <a href="/export">Click here to download the generated transcript</a>
    </body>
</html>

Writing templates/index.html


### 1.3 Setup Backend and Deploy it using Flask

In [ ]:
# import important libraries and modules
import os
import torch
import torchaudio

from flask import Flask, render_template, request, redirect, url_for, Response
from flask_ngrok import run_with_ngrok 

# create flask instance and host it using ngrok
app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = '.'
run_with_ngrok(app) 

# create function for handling home page request
@app.route('/', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        file = request.files['file']

        if file:
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], 'video.mp4'))
            return redirect(url_for('upload_file', filename=file.filename))
    return render_template("index.html")

# create function for transcript generation
@app.route("/export")
def export():
    # download model from torch hub
    model, decoder, utils = torch.hub.load('snakers4/silero-models', model='silero_stt', language='en')

    # import required modules/utilities
    (read_batch, split_into_batches, read_audio, prepare_model_input) = utils
    
    # extract audio from the video
    os.system("ffmpeg -i 'video.mp4' -vn -acodec copy audio.aac")
    os.system("ffmpeg -i audio.aac audio.wav")

    # preprocess input audio
    test_files = ['audio.wav']
    batches = split_into_batches(test_files, batch_size=10)
    input = prepare_model_input(read_batch(batches[0]))

    # send input to model to get prediction
    text = ""
    output = model(input)
    for example in output:
        pred = decoder(example.cpu())
        text = text + pred
        
    # return file to user in a downloadable format    
    return Response(
        text,
        mimetype="text/txt",
        headers={
            "Content-disposition": "attachment; filename=transcript.txt"
        }
    )
		
# run flask app
if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://bca09d4bbc32.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [01/Apr/2021 12:05:17] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Apr/2021 12:05:18] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [01/Apr/2021 12:06:04] "POST / HTTP/1.1" 302 -
127.0.0.1 - - [01/Apr/2021 12:06:04] "GET /?filename=video.mp4 HTTP/1.1" 200 -
Downloading: "https://github.com/snakers4/silero-models/archive/master.zip" to /root/.cache/torch/hub/master.zip


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:889: UserWarning: stft will soon require the return_complex parameter be given for real inputs, and will further require that return_complex=True in a future PyTorch release. (Triggered internally at  /pytorch/aten/src/ATen/native/SpectralOps.cpp:639.)
  result = self.forward(*input, **kwargs)
127.0.0.1 - - [01/Apr/2021 12:06:56] "GET /export HTTP/1.1" 200 -
